# HW 2: Corporate Bond Pricing (due by 9.21 Fri)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2018`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2018/HW2/HW2.ipynb`__  (Please use the same name for repository and ipynb file)
* Adding more code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [59]:
import numpy as np

In [6]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [35]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.097610156870855, 10.0)

In [61]:
# Put your code here to price the corporate bond

def corp_bond(mat=10.0, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=10000):
    
    # generate exponential dist. RV defaul time everytime you calculate the price
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    
    # use "price" to remember the result you get in each MC calculation
    price=np.zeros(n_sample)
    
    # calculate the prices of the bond in 10000 random cases and use the mean as the final price
    for t in range (0,n_sample):
        if default_time[t]<= mat:    #default case
            price[t]=recovery/((1+rf_rate)**default_time[t])
        else:                        #undefault case
            price[t]=1/(1+rf_rate)**mat
            
    ### <--
    ### <--
    return np.mean(price)

# Call your function
print(corp_bond())

# Find the mean and std by calling the function 100 times.
bondprice=np.zeros(100)
for i in range(0,100):
    bondprice[i]=corp_bond()

print(np.mean(bondprice),np.std(bondprice))

0.4414044377317106
0.4422779494767182 0.002468032904703663


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [11]:
# For example, antithetic method means
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.999999999999998, 10.0)

In [66]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=10000, antithetic=True, mean_match=True):
    ### get the sample using different methods
    if(antithetic):        # with antithetic method
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        if(mean_match):   #add mean_match method
            default_time += 1/def_rate-default_time.mean()
            
    else:                 #without antithetic method
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        if(mean_match):   #add mean_match method
            default_time += 1/def_rate-default_time.mean()
            
    # get the number of the new sample
    n_new_sample=len(default_time)
    
    # use "price" to remember the result you get in each MC calculation
    price=np.zeros(n_new_sample)
    
    # calculate the prices of the bond in 10000 random cases and use the mean as the final price
    for t in range (0,n_new_sample):
        if default_time[t]<= mat:    #default case
            price[t]=recovery/((1+rf_rate)**default_time[t])
        else:                        #undefault case
            price[t]=1/(1+rf_rate)**mat                                   
    ### <--
    return np.mean(price)

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
bondprice1=np.zeros(100)
bondprice2=np.zeros(100)
bondprice3=np.zeros(100)
for i in range(0,100):
    bondprice1[i]=corp_bond_cv(mean_match=False)   # case(i)
    bondprice2[i]=corp_bond_cv(antithetic=False)   # case(ii)
    bondprice3[i]=corp_bond_cv()                   # case(iii)

print(np.mean(bondprice1),np.std(bondprice1))
print(np.mean(bondprice2),np.std(bondprice2))
print(np.mean(bondprice3),np.std(bondprice3))

0.4418172140932333 0.0013462801770269335
0.44207094838071403 0.0015512834309549409
0.4420006311200233 0.0011563719919209074


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [65]:
### Put the analytic expression for the corporate bond price
def corp_bond_accu(mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3):
    ### <--
    alpha=1/(np.exp(def_rate))/(1+rf_rate)
    
    price=def_rate*recovery*(alpha**mat-1)/(np.log(alpha))+alpha**mat
    ### <--
    ### <--
    return price

corp_bond_accu()

0.4419067811863257